# Computational Tests

This notebook is intended for running different kinds of analyses that would validate our reconstruction.

## 1. Subsystem Overview and Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from google_sheet import GoogleSheet

In [ ]:
##### ----- Generate Subsystems datasets from Google Sheet ----- #####

#Credential file
KEY_FILE_PATH = 'credentials.json'

#CHO Network Reconstruction + Recon3D_v3 Google Sheet ID
SPREADSHEET_ID = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_subsystems = 'Summary Systems'
subsystems = sheet.read_google_sheet(sheet_subsystems)

In [ ]:
#### --- Pie Chart of the Amount of Reactions per System --- ####

subsystems["Number of Reactions"] = pd.to_numeric(subsystems["Number of Reactions"])

# Aggregate data by "System" and sum "Number of Reactions"
system_reactions = subsystems.groupby("System")["Number of Reactions"].sum()

# Set the figure and axes for more control
fig, ax = plt.subplots(figsize=(10, 7))

# Generate the pie chart again with the custom labels
patches, texts, autotexts = ax.pie(system_reactions, labels=system_reactions.index, autopct=lambda p: '{:.1f}%'.format(p) if p > 0 else '', startangle=140, colors=plt.cm.tab20.colors)

# Improve aesthetics
for text in texts + autotexts:
    text.set_color('black')
ax.set_title("Pie Chart of Number of Reactions by System", pad=30)  # Move the title upwards by increasing pad

# Set equal aspect ratio
ax.axis('equal')

# Save the plot
plt.savefig('../Plots/pie_chart_reactions_per_system.png')

# Display the plot
plt.show()

In [ ]:
# Generate the Sunburst Plot
fig = px.sunburst(subsystems, path=['System', 'Subsystems','Kegg Pathway'],
                  color='System', title='System and Subsystems Relationship')

fig.update_layout(width=1200, height=1000)

fig.write_html("../Plots/sunburst_subsystems.html") # Save as interactive HTML file
fig.write_image("../Plots/sunburst_subsystems.png", width=1200, height=1000)
# Show the plot
fig.show()

#### Auxotrophies

In [ ]:
from cobra.io import load_json_model
iCHO_path = "iCHOv3_CHO_04032024.json"
iCHO = load_json_model(iCHO_path)

In [ ]:
amino_acids = {
    "arginine": ["EX_arg_L_e", "EX_arg_D_e"],
    "asparagine": ["EX_asn_L_e", "EX_asn_D_e"],
    "cysteine": ["EX_cys_L_e", "EX_cys_D_e", "EX_Lcystin_e"],
    "histidine": ["EX_his_L_e", "EX_his_D_e"],
    "isoleucine": ["EX_ile_L_e", "EX_ile_D_e"],
    "leucine": ["EX_leu_L_e", "EX_leu_D_e"],
    "lysine": ["EX_lys_L_e", "EX_lys_D_e"],
    "methionine": ["EX_met_L_e", "EX_met_D_e"],
    "phenylalanine": ["EX_phe_L_e", "EX_phe_D_e"],
    "proline": ["EX_pro_L_e", "EX_pro_D_e"],
    "threonine": ["EX_thr_L_e", "EX_thr_D_e"],
    "tryptophan": ["EX_trp_L_e", "EX_trp_D_e"],
    "valine": ["EX_val_L_e", "EX_val_D_e"]
}
for amino_acid in amino_acids:
    # ----- Setup initial bounds -----
    for exchange_reaction in iCHO.exchanges:
        exchange_reaction.bounds = -10, 10

    # Arginine
    iCHO.reactions.get_by_id('GAUGE-R00557').bounds = 0, 10
    iCHO.reactions.get_by_id('GAUGE-R10107').bounds = 0, 0
    iCHO.reactions.get_by_id('GAUGE-R00558').bounds = 0, 0
    iCHO.reactions.get_by_id('GLYAMDTRc').bounds = 0, 0
    iCHO.reactions.get_by_id('GAUGE-R10107').bounds = 0, 0
    iCHO.reactions.get_by_id('EX_valarggly_e').bounds = 0, 10
    iCHO.reactions.get_by_id('ARGSL').bounds = 0, 0

    # Asparigine
    iCHO.reactions.get_by_id('ASNS1').bounds = 0, 0

    # Cysteine
    iCHO.reactions.get_by_id('r0129').bounds = 0, 0
    iCHO.reactions.get_by_id('EX_cgly_e').bounds = 0, 0
    iCHO.reactions.get_by_id('AMPTASECG').bounds = -10, 0
    iCHO.reactions.get_by_id('AMPTASECGe').bounds = -10, 0
    iCHO.reactions.get_by_id('CYSTGL').bounds = -10, 0
    iCHO.reactions.get_by_id('EX_HC00250_e').bounds = 0, 0
    iCHO.reactions.get_by_id('EX_sfcys_e').bounds = 0, 0

    # Histidine
    iCHO.reactions.get_by_id('VALTRPVALr').bounds = 0,0 

    # Isoleucine - DONE
    iCHO.reactions.get_by_id('EX_CE2916_e').bounds = 0,0 
    iCHO.reactions.get_by_id('EX_CE2915_e').bounds = 0,0 
    iCHO.reactions.get_by_id('ILETA').bounds = 0, 10
    iCHO.reactions.get_by_id('ILETAm').bounds = 0, 10

    # Leucine
    iCHO.reactions.get_by_id('LEULEULAPc').bounds = 0,0 
    iCHO.reactions.get_by_id('EX_leugly_e').bounds = 0,0 
    iCHO.reactions.get_by_id('EX_glyleu_e').bounds = 0,0 
    iCHO.reactions.get_by_id('LEUTA').bounds = 0, 10 
    iCHO.reactions.get_by_id('LEUTAm').bounds = 0, 10 
    iCHO.reactions.get_by_id('EX_CE5797_e').bounds = 0, 0

    # Lysine
    iCHO.reactions.get_by_id('EX_biocyt_e').bounds = 0,0 

    # Methionine
    iCHO.reactions.get_by_id('METS').bounds = -10, 0 
    iCHO.reactions.get_by_id('BHMT').bounds = -10, 0 
    iCHO.reactions.get_by_id('GAUGE-R00648').bounds = 0, 10 
    iCHO.reactions.get_by_id('UNK2').bounds = -10, 0 
    iCHO.reactions.get_by_id('UNK3').bounds = -10, 0 
    iCHO.reactions.get_by_id('TYRA').bounds = -10, 0 
    #iCHO.reactions.get_by_id('GAUGE-R06895').bounds = 0, 0 # Curated by MR with 1 score / Erased from the reconstruction

    # Phenylalanine
    iCHO.reactions.get_by_id('EX_CE5786_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_pheleu_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_glyphe_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_CE2917_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_CE5786_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_CE5789_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_phpyr_e').bounds = 0, 0 

    # Proline
    iCHO.reactions.get_by_id('EX_glypro_e').bounds = 0, 10
    iCHO.reactions.get_by_id('EX_progly_e').bounds = 0, 10
    iCHO.reactions.get_by_id('P5CR').bounds = 0, 0
    iCHO.reactions.get_by_id('P5CRxm').bounds = 0, 0
    iCHO.reactions.get_by_id('P5CRx').bounds = 0, 0
    iCHO.reactions.get_by_id('P5CRm').bounds = 0, 0
    iCHO.reactions.get_by_id('r1453').bounds = 0, 10

    # Threonine
    #iCHO.reactions.get_by_id('THRS').bounds = 0,0  # Erased from the reconstruction


    # Tryptophan
    iCHO.reactions.get_by_id('NBAHH_ir').bounds = 0,0  

    # Valine
    iCHO.reactions.get_by_id('EX_valarggly_e').bounds = 0, 10
    iCHO.reactions.get_by_id('EX_vallystyr_e').bounds = 0, 10
    iCHO.reactions.get_by_id('VALTA').bounds = 0, 10
    iCHO.reactions.get_by_id('VALTAm').bounds = 0, 10
    iCHO.reactions.get_by_id('EX_valval_e').bounds = 0, 10
    # -----------------------------------------

    for exchange_reaction in iCHO.exchanges:
        if exchange_reaction.id in amino_acids[amino_acid]:
            exchange_reaction.bounds = 0, 10
            # print(amino_acid, exchange_reaction.id, amino_acids[amino_acid])
    sol = iCHO.optimize()
    print(amino_acid, sol.objective_value)


In [ ]:
amino_acids = {
    "alanine": ["EX_ala_B_e", "EX_ala_L_e", "EX_ala_D_e"],
    "arginine": ["EX_arg_L_e", "EX_arg_D_e"],
    "asparagine": ["EX_asn_L_e", "EX_asn_D_e"],
    "aspartic acid": ["EX_asp_L_e", "EX_asp_D_e"],
    "cysteine": ["EX_cys_L_e", "EX_cys_D_e"],
    "glutamine": ["EX_gln_L_e", "EX_gln_D_e"],
    "glutamic acid": ["EX_glu_L_e", "EX_glu_D_e"],
    "glycine": ["EX_gly_e"],
    "histidine": ["EX_his_L_e", "EX_his_D_e"],
    "isoleucine": ["EX_ile_L_e", "EX_ile_D_e"],
    "leucine": ["EX_leu_L_e", "EX_leu_D_e"],
    "lysine": ["EX_lys_L_e", "EX_lys_D_e"],
    "methionine": ["EX_met_L_e", "EX_met_D_e"],
    "phenylalanine": ["EX_phe_L_e", "EX_phe_D_e"],
    "proline": ["EX_pro_L_e", "EX_pro_D_e"],
    "serine": ["EX_ser_L_e", "EX_ser_D_e"],
    "threonine": ["EX_thr_L_e", "EX_thr_D_e"],
    "tryptophan": ["EX_trp_L_e", "EX_trp_D_e"],
    "tyrosine": ["EX_tyr_L_e", "EX_tyr_D_e"],
    "valine": ["EX_val_L_e", "EX_val_D_e"]
}

def test_auxotrophy(model, amino_acids):
    # Iterate through each amino acid and its corresponding exchange reactions
    for amino_acid, reactions in amino_acids.items():
        # Set the bounds for each exchange reaction
        for exchange in model.exchanges:
            if exchange.id in reactions:
                exchange.bounds = (0, 10)  # Allow secretion but not uptake
            else:
                exchange.bounds = (-10, 10)  # Allow uptake and secretion

        # Optimize the model for the current setup
        solution = model.optimize()
        if solution.status == "optimal":
            print(f"{amino_acid} biomass production: {solution.objective_value}")
        else:
            print(f"{amino_acid} optimization failed: {solution.status}")

In [ ]:
test_auxotrophy(iCHO, amino_acids)

In [ ]:
iCHO.metabolites.leu_L_c.summary()

#### Matrix Sparsity

#### Gene Deletion test

##### Essential for growth genes and comparison to (An optimised genome-wide, virus free CRISPR screen for mammalian cells)

##### Commparsion to other models

#### Biomass prediction using exp. data (TODO)

Incorporate uptake and secretion rates from (https://www.sciencedirect.com/science/article/pii/S2405471216303635?via%3Dihub) data and calculate the growth rate. Try ecFBA with enzyme trunover numbers. Reproduce Hooman's analsys with iCHO_v1.

In [1]:
import pandas as pd
import cobra
from cobra.io import load_json_model

In [2]:
model = load_json_model('iCHOv3_CHO_05032024.json')

In [3]:
# model.reactions.biomass_cho.bounds = (0.033,0.033)
model.reactions.biomass_cho.bounds = (0.0,10)
model.reactions.biomass_cho_prod.bounds = (0.0,10)

bio_index = 0
bio_prod_index = 0
for counter, rxn in enumerate(model.reactions):
    if rxn.id == 'biomass_cho':
        bio_index = counter
    elif rxn.id == 'biomass_cho_prod':
        bio_prod_index = counter
print(bio_index, bio_prod_index)

214 215


In [4]:
uptsec = pd.read_excel('../Data/Uptake_Secretion_Rates/mmc4.xlsx', header=2)
biomass_bounds = uptsec[42:44]
uptsec = uptsec[:-5]

In [23]:
uptsec_dict = uptsec.set_index('Reaction')['HP'].to_dict()
uptsec_dict

{'DM_igg_g': 2.02422458111942e-05,
 'EX_ac_e': 7.38007e-05,
 'EX_ala_L_e': 0.023726937,
 'EX_arg_L_e': -0.01103321,
 'EX_asn_L_e': -0.074723247,
 'EX_asp_L_e': -0.025166052,
 'EX_chol_e': -0.003099631,
 'EX_cit_e': 0.002214022,
 'EX_cys_L_e': -0.00521978,
 'EX_for_e': 0.012583026,
 'EX_glc_e': -0.271070111,
 'EX_gln_L_e': 0.002767528,
 'EX_glu_L_e': -0.012398524,
 'EX_gly_e': 0.00697417,
 'EX_glyc_e': 0.015129151,
 'EX_his_L_e': -0.006457565,
 'EX_ile_L_e': -0.011808118,
 'EX_lac_L_e': 0.135608856,
 'EX_leu_L_e': -0.020184502,
 'EX_lys_L_e': -0.012583026,
 'EX_mal_L_e': 0.001660517,
 'EX_met_L_e': -0.004797048,
 'EX_nh4_e': 0.043763838,
 'EX_o2_e': -1.127472527,
 'EX_phe_L_e': -0.006457565,
 'EX_pro_L_e': -0.011623616,
 'EX_pyr_e': -0.010774908,
 'EX_ser_L_e': -0.043062731,
 'EX_succ_e': 0.000405904,
 'EX_thr_L_e': -0.015313653,
 'EX_trp_L_e': -0.003173432,
 'EX_tyr_L_e': -0.005276753,
 'EX_val_L_e': -0.015092251,
 'EX_h2o_e': -1000.0,
 'EX_hco3_e': -1000.0,
 'EX_pi_e': -1000.0,
 'EX_s

In [24]:
for rxn in model.boundary:

    # IMPORTANT, all ex should be 0 - 1000, except those that we know are being uptaken by the cell
    
    if rxn.id.startswith("EX_"):
        rxn.bounds = (0,1000) 
    if rxn.id.startswith("SK_"):
        rxn.bounds = (-1000,1000)
    if rxn.id.startswith("DM_"):
        rxn.bounds = (0,1000)

In [25]:
for reaction in model.reactions:
    for r,v in uptsec_dict.items():
        if reaction.id == r:
            print(f'{reaction.reaction}, Original Bound: {reaction.lower_bound}, New Uptake/Secretion Rate: {v}')
            reaction.lower_bound = v

ac_e --> , Original Bound: 0, New Uptake/Secretion Rate: 7.38007e-05
ala_L_e --> , Original Bound: 0, New Uptake/Secretion Rate: 0.023726937
arg_L_e --> , Original Bound: 0, New Uptake/Secretion Rate: -0.01103321
asn_L_e --> , Original Bound: 0, New Uptake/Secretion Rate: -0.074723247
asp_L_e --> , Original Bound: 0, New Uptake/Secretion Rate: -0.025166052
cit_e --> , Original Bound: 0, New Uptake/Secretion Rate: 0.002214022
cys_L_e --> , Original Bound: 0, New Uptake/Secretion Rate: -0.00521978
for_e --> , Original Bound: 0, New Uptake/Secretion Rate: 0.012583026
glc_D_e --> , Original Bound: 0, New Uptake/Secretion Rate: -0.271070111
gln_L_e --> , Original Bound: 0, New Uptake/Secretion Rate: 0.002767528
glu_L_e --> , Original Bound: 0, New Uptake/Secretion Rate: -0.012398524
gly_e --> , Original Bound: 0, New Uptake/Secretion Rate: 0.00697417
glyc_e --> , Original Bound: 0, New Uptake/Secretion Rate: 0.015129151
h2o_e --> , Original Bound: 0, New Uptake/Secretion Rate: -1000.0
hco3_

In [26]:
display(biomass_bounds)

model.objective = model.reactions.biomass_cho_prod
pfba_solution = cobra.flux_analysis.pfba(model)
pfba_solution
print("Biomass_prod: ", pfba_solution[bio_prod_index])

model.objective = model.reactions.biomass_cho
pfba_solution = cobra.flux_analysis.pfba(model)
pfba_solution
print("Biomass: ", pfba_solution[bio_index])


,Reaction,HP,HP+NaBu,LP,LP+NaBu,Early Exp,Late Exp,Cold 1,Cold 2
42,Predicted growth rate,0.021629,0.020189,0.014507,0.028659,0.022871,0.009696,0.019023,0.033715
43,Experimental growth rate,0.030608,0.024192,0.038401,0.028251,0.024747,0.011613,0.025397,0.023558


Biomass_prod:  0.027861829083968927
Biomass:  0.030412260543272358


In [34]:
c=0
for f in pfba_solution.fluxes:
    if f != 0:
        c+=1
        
print(c)

589


In [47]:
for r,f in pfba_solution.fluxes.items():
    if f != 0:
        if r.startswith(('EX_','SK_')):
            print(r,f)

SK_23camp_l -0.0048691130621546395
EX_xol7ah3_e -6.308512385996725e-54
EX_3mob_e 1.3473953917553107e-20
EX_ac_e 7.38007e-05
EX_ala_L_e 0.015793358
EX_anth_e 0.0022555856829044514
EX_arg_L_e -0.006002937088494741
EX_asn_L_e -0.005113822395835127
EX_asp_L_e -0.016595251669876163
EX_C02528_e 1.5696007330743604e-20
EX_CE4633_e -7.56687732141116e-19
EX_cit_e 0.00195572
EX_co2_e 0.01811715612751126
EX_crn_e 4.297446475729146e-18
EX_cspg_a_e 1.1455892335256405e-35
EX_cys_L_e -0.002102434715937869
EX_dchac_e 3.610057048572719e-18
EX_estradiolglc_e -1.353956488023935e-18
EX_fol_e 1.3539564880239347e-18
EX_for_e 0.011623616
EX_glcur_e -3.221969719290864e-35
EX_gln_L_e 0.000332103
EX_glu_L_e -0.013247232
EX_gly_e 0.007343173
EX_glyc_e 0.015387454
EX_h_e 0.0379498380637018
EX_h2o2_e 0.0048691130621546395
EX_HC00250_e 4.295959625721152e-35
EX_HC02180_e -1.8234934390529962e-19
EX_his_L_e -0.003726937
EX_ile_L_e -0.0037273985621098446
EX_lac_L_e 0.134391144
EX_leu_L_e -0.019520295
EX_leuktrB4_e -1.56